# Define imports, etc :


In [ ]:

import json
import ijson
import pycurl
import time
import numpy as np
import os

#from pyeemd import ceemdan
#from pyeemd.utils import plot_imfs

try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO

MASTER_URL ='http://Node-LB-618209181.us-east-1.elb.amazonaws.com/'
DEVLIST_URL = '1/Devices'
SRC_PATH = './JSON_DIR/'
DES_PATH = './DATA_DIR/'
TODAY_ONLY = 0
TAR_PTS = 1000

if not os.path.exists(DES_PATH):
	try:
    	        os.makedirs(DES_PATH)
	except:
		pass

def getBody(url):
        buffer = BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, MASTER_URL+url)
        c.setopt(c.WRITEDATA, buffer)
        c.perform()
        x = buffer.getvalue()
        c.close()
        #return buffer.getvalue()
        return x

def parse_file(filename):
    with open(filename, 'r') as f:
        z=f.readlines()
        arr = np.array([elem[:-1].split(',') for elem in z])
        d = {}
        d["tem"] = arr[:,0].astype(float)
        d["hum"] = arr[:,1].astype(float)
        d["co2"] = arr[:,2].astype(float)
        d["dewpt"] = arr[:,3].astype(float)
        d["pm"] = arr[:,4].astype(float)
        ts = arr[:,5]
        ts_mod = [time.strptime(tt, "%Y-%m-%dT%H:%M:%S.000Z") for tt in ts]
        d["upd_ts"] = [np.uint32(time.mktime(x)) for x in ts_mod]
        d["tod"] = np.array([x.tm_hour*60 + x.tm_min for x in ts_mod])#tod= time_of_day
        d["dow"] = np.array([x.tm_wday for x in ts_mod])               #day_of_week
        d["voc"] = arr[:,6].astype(float)
        #return tem, hum, co2, dewpt, pm, upd_ts, tod, dow, voc
        return d    

#all_devs = json.loads(getBody(DEVLIST_URL))
#dev_list = [dev[u'device'] for dev in all_devs]
#name_list = [dev[u'name'] for dev in all_devs]
json_list =  os.listdir(SRC_PATH)
d_list = []	#device list, with details.

#jf--> json file
for idx, jf in enumerate(json_list):
	temp_list = []
	hum_list =[]
	co2_list = []
	dewpt_list = []
	pm_list = []
	ts_list = [] #timestamp
	voc_list = []
	print(jf)
	print("Processing %d/%d..." %(idx+1, len(json_list)))
	dev = jf.split('_')[0]
	name = jf.split('_')[1][:-5]
	print(name)
	start = time.time()
	try:
	        with open(SRC_PATH+jf, 'r') as f:
                        parser = ijson.parse(f)
                        for prefix, event, value in parser:
                                if prefix.endswith('temperature'):
                                        temp_list.append(value*1.8/1000.0 + 32)
                                elif prefix.endswith('humidity'):
                                        hum_list.append(value/1000.0)
                                elif prefix.endswith('co2'):
                                        co2_list.append(value)
                                elif prefix.endswith('dewpoint'):
                                        dewpt_list.append(value)
                                elif prefix.endswith('pm'):
                                        pm_list.append(value)
                                elif prefix.endswith('timestamp'):
                                        ts_list.append(value)
                                elif prefix.endswith('voc'):
                                        voc_list.append(value)
                        ## Saved as Temp | Hum | Co2 | Dewpt | pm | ts | VOC
                        arr = np.vstack((temp_list, hum_list)) 
                        arr = np.vstack((arr, co2_list))
                        arr = np.vstack((arr, dewpt_list))
                        arr = np.vstack((arr, pm_list))
                        arr = np.vstack((arr, ts_list))
                        arr = np.vstack((arr, voc_list))
                        arr = arr.T
                        np.savetxt(DES_PATH + dev + '_' + name + '.txt', arr[::100,:], fmt = '%s', delimiter=',')

	except Exception as e:
		print("Error: "+str(e))
		pass
                
	print("Processed & Saved in %d seconds\n" %(time.time()-start))
print("Parse process Completed.")

0xf8f005f0270d_Wei_s Apartment.json
Processing 1/12...

Processed & Saved in 56 seconds

0xf8f005f066a3_Atelier Ten.json
Processing 2/12...
Atelier Ten


In [5]:
# Data Visualization Libraries
%matplotlib inline

import matplotlib as mpl
#mpl.use('agg')
mpl.rcParams['axes.unicode_minus']=False
mpl.rcParams['text.usetex'] = False
mpl.rcParams['text.latex.unicode']=False
#mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#%matplotlib inline
import seaborn as sns


## Numerical Analysis Libraries
import numpy as np
from numpy.fft import fft as fft
from scipy.stats.stats import pearsonr
import sklearn as sl
import pandas as pd

#import pyeemd
#from pyeemd.utils import plot_imfs


## Generic Python Libraries:
import os
import time
from datetime import datetime as dt
import decimal


In [6]:
# return a sorted copy of an array
txt_list = np.sort(os.listdir('./DATA_DIR/')) 
sns.set(font="sans-serif")

print (txt_list)


['0xf8f005f0270d_.txt' '0xf8f005f066a3_Atelier Ten.txt'
 '0xf8f005f1c71a_Civic Hall Boffi.txt' '0xf8f005f1c71c_Studios Paris.txt'
 '0xf8f005f2f32e_CASE #3.txt' '0xf8f005f2f3f3_Civic Hall.txt'
 '0xf8f005f417ef_CASE #2.txt' '0xf8f005f41863_Simone Bedroom.txt'
 '0xf8f005f41877_Ju.txt' '0xf8f005f41961_CCS.txt'
 '0xf8f005f41991_CASE (Anna Dyson Office).txt']


In [7]:
def parse_file(filename):
    with open(filename, 'r') as f:
        z=f.readlines()
        
        #print "f.readlines(): \n" + str(z)
        
        arr = np.array([elem[:-1].split(',') for elem in z])
        
        #print "\narr: \n"
        #print arr

        d = {}
        try:
            voc_list = arr[:,6].astype(float)
        except:
            arr = np.array([elem[:-1].split(',') for elem in z[:-1]])
            voc_list = arr[:,6].astype(float)
        low_thresh = 124
        high_thresh = 1e6
                
        d["voc"] = voc_list[(voc_list>low_thresh) & (voc_list<high_thresh)]
        
        #print "\nvoc_list length: " + str(len(voc_list))
        #print voc_list
        
        #print "\nd[voc] length: " + str(len(d["voc"]))
        #print d["voc"]
        
        d["tem"] = arr[:,0].astype(float)[(voc_list>low_thresh) & (voc_list<high_thresh)]
        d["hum"] = arr[:,1].astype(float)[(voc_list>low_thresh) & (voc_list<high_thresh)]
        d["co2"] = arr[:,2].astype(float)[(voc_list>low_thresh) & (voc_list<high_thresh)]
        d["dewpt"] = arr[:,3].astype(float)[(voc_list>low_thresh) & (voc_list<high_thresh)]
        
        d["pm"] = arr[:,4].astype(float)[(voc_list>low_thresh) & (voc_list<high_thresh)]
        d["pm"] = d["pm"]*0.103 - 0.698 #in ug/m3
        
        ts = arr[:,5][(voc_list>low_thresh) & (voc_list<high_thresh)]
        ts_mod = [time.strptime(tt, "%Y-%m-%dT%H:%M:%S.000Z") for tt in ts]
        d["upd_ts"] = [np.uint32(time.mktime(x)) for x in ts_mod]
        d["tod"] = np.array([x.tm_hour*60 + x.tm_min for x in ts_mod])#tod= time_of_day
        d["dow"] = np.array([x.tm_wday for x in ts_mod])               #day_of_week
        d["time_date"] = np.array([dt.utcfromtimestamp(tim).strftime("%Y-%m-%d") for tim in d["upd_ts"]])
        
        #print "\nd[tem] length: " + str(len(d["tem"]))
        #print 
        #return tem, hum, co2, dewpt, pm, upd_ts, tod, dow, voc
        return d
        d["tem"]
        


In [8]:
d = parse_file("./DATA_DIR/"+txt_list[10])
print (d['tem']) 
print (d['hum']) 


[ 84.3026  82.121   82.9814 ...,  77.9918  78.1268  77.8712]
[ 34.899  19.024  18.156 ...,  42.091  42.25   42.175]


In [9]:
def cor_mat_print(d):
    x = np.sort(d.keys())
    
    #print(d[x[0]])
    #print(d[x[2]])
    print (x)
    
    l = len(x)
    cor_mat = np.zeros((l,l))
    #print cor_mat
    for i in range(0,l):
        for j in range(0,l):
            np.set_printoptions(suppress=True, precision=4)
            #decimal.getcontext().prec=5
            feat1 = d[x[i]]
            feat2 = d[x[j]]
            try:
                cor_mat[i][j] = pearsonr(feat1, feat2)[0]
            #print pearsonr(feat1, feat2)[0]
            except:
                cor_mat[i][j] = np.nan
    #print x
    #print cor_mat#.astype(np.uint32)#*1.0/np.sum(cor_mat.astype(np.float32))
    return cor_mat, x


In [10]:
d = parse_file("./DATA_DIR/"+txt_list[5])
#print d
cor_mat_print(d)
          

AxisError: axis -1 is out of bounds for array of dimension 0

In [ ]:
d = parse_file("./DATA_DIR/"+txt_list[5])

In [ ]:
cor_mat_print(d)

In [ ]:
def cor_mat_print(d):
    x = np.sort(list(d.keys()))
    
    #print(d[x[0]])
    #print(d[x[2]])
    print (x)
    
    l = len(x)
    cor_mat = np.zeros((l,l))
    #print cor_mat
    for i in range(0,l):
        for j in range(0,l):
            np.set_printoptions(suppress=True, precision=4)
            #decimal.getcontext().prec=5
            feat1 = d[x[i]]
            feat2 = d[x[j]]
            try:
                cor_mat[i][j] = pearsonr(feat1, feat2)[0]
            #print pearsonr(feat1, feat2)[0]
            except:
                cor_mat[i][j] = np.nan
    #print x
    #print cor_mat#.astype(np.uint32)#*1.0/np.sum(cor_mat.astype(np.float32))
    return cor_mat, x


In [ ]:
d = parse_file("./DATA_DIR/"+txt_list[5])

In [ ]:
cor_mat_print(d)

In [ ]:
d.keys()

In [ ]:
c_list = []
d_list = []
for filename in txt_list:
    #print filename
    d = parse_file("./DATA_DIR/"+filename)
    d_list.append(d)
    c,x  = cor_mat_print(d)
    c_list.append(c)

#print c_list[2]
print (d_list[10])


# Assignment-04 practice

# Graph of avg_VOC

In [ ]:
d = parse_file("./DATA_DIR/"+txt_list[0])

In [ ]:
df = pd.DataFrame({'my_dates':d['time_date'],'voc':d['voc']})
df['my_dates'] = pd.to_datetime(df['my_dates'])
df['day_of_week'] = df['my_dates'].dt.weekday_name
tuesday=df.loc[df['day_of_week'] == 'Tuesday']
np.average(tuesday['voc'])

In [ ]:
df = pd.DataFrame({'my_dates':d['time_date'],'voc':d['voc']})
df

In [ ]:
df['my_dates'] = pd.to_datetime(df['my_dates'])
df['my_dates']

In [ ]:
df['day_of_week'] = df['my_dates'].dt.weekday_name
df['day_of_week']

In [ ]:
tuesday=df.loc[df['day_of_week'] == 'Tuesday']
tuesday

In [ ]:
result=np.average(tuesday['voc'])
result

In [ ]:
import plotly.plotly as py

In [ ]:
import plotly.graph_objs as go
import numpy as np

In [ ]:
N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N)+5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N)-5

In [ ]:
trace2 = go.Scatter(
    x = ['s','m'],
    y = [result,10],
    mode = 'lines+markers',
    name = 'lines+markers'
)

In [ ]:
data = [trace2]

In [ ]:
py.iplot(data, filename='line-mode')

In [ ]:
trace1 = go.Scatter(
    x=['s', 'm'],
    y=[result,2],
    name = 'Gaps', # Style name/legend entry with html tags
    connectgaps=False
)

In [ ]:
sunday=df.loc[df['day_of_week'] == 'Sunday']
monday=df.loc[df['day_of_week'] == 'Monday']
tuesday=df.loc[df['day_of_week'] == 'Tuesday']
wednesday=df.loc[df['day_of_week'] == 'Wednesday']
thuresday=df.loc[df['day_of_week'] == 'Thursday']
friday=df.loc[df['day_of_week'] == 'Friday']
saturday=df.loc[df['day_of_week'] == 'Saturday']
sundayval=np.average(sunday['voc'])
mondayval=np.average(monday['voc'])
tuesdayval=np.average(tuesday['voc'])
wednesdayval=np.average(wednesday['voc'])
thuresdayval=np.average(thuresday['voc'])
fridayval=np.average(friday['voc'])
saturdayval=np.average(saturday['voc'])

print(sundayval)
print(mondayval)
print(tuesdayval)
print(wednesdayval)
print(thuresdayval)
print(fridayval)
print(saturdayval)

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
trace2 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval, mondayval, tuesdayval, wednesdayval, thuresdayval, fridayval, saturdayval],
  mode = 'lines+markers',
  name = 'lines+markers'
   
)
data = [trace2]
py.iplot(data, filename='line-mode')



# Graph avg_PM 

In [ ]:
df = pd.DataFrame({'my_dates':d['time_date'],'PM':d['pm']})
df

In [ ]:
df['my_dates'] = pd.to_datetime(df['my_dates'])
df['my_dates']

In [ ]:
df['day_of_week'] = df['my_dates'].dt.weekday_name
df['day_of_week']

In [ ]:
df = pd.DataFrame({'my_dates':d['time_date'],'PM':d['pm'],'days':df['day_of_week']})
df

In [ ]:
result=np.average(df['PM'])
result

In [ ]:
sunday=df.loc[df['days'] == 'Sunday']
monday=df.loc[df['days'] == 'Monday']
tuesday=df.loc[df['days'] == 'Tuesday']
wednesday=df.loc[df['days'] == 'Wednesday']
thuresday=df.loc[df['days'] == 'Thursday']
friday=df.loc[df['days'] == 'Friday']
saturday=df.loc[df['days'] == 'Saturday']
sundayval=np.average(sunday['PM'])
mondayval=np.average(monday['PM'])
tuesdayval=np.average(tuesday['PM'])
wednesdayval=np.average(wednesday['PM'])
thuresdayval=np.average(thuresday['PM'])
fridayval=np.average(friday['PM'])
saturdayval=np.average(saturday['PM'])

print(sundayval)
print(mondayval)
print(tuesdayval)
print(wednesdayval)
print(thuresdayval)
print(fridayval)
print(saturdayval)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
trace3 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval, mondayval, tuesdayval, wednesdayval, thuresdayval, fridayval, saturdayval],
  mode = 'lines+markers',
  name = 'lines+markers'
   
)
data = [trace3]
py.iplot(data, filename='line-mode')

In [ ]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 2)

fig['layout'].update(height=400, width=1100, title='avg_VOC array & avg_PM array')
py.iplot(fig, filename='simple-subplot')

In [18]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x=[2, 3.5, 6],
    y=[1, 1.5, 1],
    text=['Vertical Line', 'Horizontal Dashed Line', 'Diagonal dotted Line'],
    mode='text',
)
data = [trace0]
layout = {
    'xaxis': {
        'range': [0, 7]
    },
    'yaxis': {
        'range': [0, 2.5]
    },
    'shapes': [
        # Line Vertical
        
        # Line Horizontal
        {
            'type': 'line',
            'x0': 0,
            'y0': 200,
            'x1': 0,
            'y1': 200,
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 4,
                'dash': 'dashdot',
            },
        },
        # Line Diagonal
        
    ]
}

fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='shapes-lines')

# Final assignmnt-04

In [40]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

d = parse_file("./DATA_DIR/"+txt_list[0])
df = pd.DataFrame({'my_dates':d['time_date'],'PM':d['pm'],'voc':d['voc']})
df['my_dates'] = pd.to_datetime(df['my_dates'])
df['days'] = df['my_dates'].dt.weekday_name


sunday=df.loc[df['days'] == 'Sunday']
monday=df.loc[df['days'] == 'Monday']
tuesday=df.loc[df['days'] == 'Tuesday']
wednesday=df.loc[df['days'] == 'Wednesday']
thuresday=df.loc[df['days'] == 'Thursday']
friday=df.loc[df['days'] == 'Friday']
saturday=df.loc[df['days'] == 'Saturday']

sundayval=np.average(sunday['PM'])
mondayval=np.average(monday['PM'])
tuesdayval=np.average(tuesday['PM'])
wednesdayval=np.average(wednesday['PM'])
thuresdayval=np.average(thuresday['PM'])
fridayval=np.average(friday['PM'])
saturdayval=np.average(saturday['PM'])

sundayval1=np.average(sunday['voc'])
mondayval1=np.average(monday['voc'])
tuesdayval1=np.average(tuesday['voc'])
wednesdayval1=np.average(wednesday['voc'])
thuresdayval1=np.average(thuresday['voc'])
fridayval1=np.average(friday['voc'])
saturdayval1=np.average(saturday['voc'])

trace1 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval1, mondayval1, tuesdayval1, wednesdayval1, thuresdayval1, fridayval1, saturdayval1],
  mode = 'lines+markers',
  name = 'avg_VOC array'
   
)
data = [trace1]


trace2 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval, mondayval, tuesdayval, wednesdayval, thuresdayval, fridayval, saturdayval],
  mode = 'lines+markers',
  name = 'avg_PM array'
   
)
data = [trace2]

tracethreshold = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y =  [200, 200, 200, 200, 200, 200, 200],
  mode = 'lines+markers',
  name = 'voc_threshold',
     line = dict(
       color = ('rgb(205, 12, 24)'),
       width = 4,
       dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)
data = [tracethreshold]

fig = tools.make_subplots(rows=1, cols=2)


fig.append_trace(tracethreshold, 1, 2)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace1, 1, 2)


fig['layout'].update(height=400, width=1100, title='avg_VOC array & avg_PM array')


py.iplot(fig, filename='simple-subplot')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [42]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

d = parse_file("./DATA_DIR/"+txt_list[1])
df = pd.DataFrame({'my_dates':d['time_date'],'PM':d['pm'],'voc':d['voc']})
df['my_dates'] = pd.to_datetime(df['my_dates'])
df['days'] = df['my_dates'].dt.weekday_name


sunday=df.loc[df['days'] == 'Sunday']
monday=df.loc[df['days'] == 'Monday']
tuesday=df.loc[df['days'] == 'Tuesday']
wednesday=df.loc[df['days'] == 'Wednesday']
thuresday=df.loc[df['days'] == 'Thursday']
friday=df.loc[df['days'] == 'Friday']
saturday=df.loc[df['days'] == 'Saturday']

sundayval=np.average(sunday['PM'])
mondayval=np.average(monday['PM'])
tuesdayval=np.average(tuesday['PM'])
wednesdayval=np.average(wednesday['PM'])
thuresdayval=np.average(thuresday['PM'])
fridayval=np.average(friday['PM'])
saturdayval=np.average(saturday['PM'])

sundayval1=np.average(sunday['voc'])
mondayval1=np.average(monday['voc'])
tuesdayval1=np.average(tuesday['voc'])
wednesdayval1=np.average(wednesday['voc'])
thuresdayval1=np.average(thuresday['voc'])
fridayval1=np.average(friday['voc'])
saturdayval1=np.average(saturday['voc'])

trace3 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval1, mondayval1, tuesdayval1, wednesdayval1, thuresdayval1, fridayval1, saturdayval1],
  mode = 'lines+markers',
  name = 'avg_VOC array'
   
)
data = [trace3]


trace4 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval, mondayval, tuesdayval, wednesdayval, thuresdayval, fridayval, saturdayval],
  mode = 'lines+markers',
  name = 'avg_PM array'
   
)
data = [trace4]

tracethreshold = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y =  [200, 200, 200, 200, 200, 200, 200],
  mode = 'lines+markers',
  name = 'voc_threshold',
     line = dict(
       color = ('rgb(205, 12, 24)'),
       width = 4,
       dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)
data = [tracethreshold]

fig = tools.make_subplots(rows=1, cols=2)


fig.append_trace(tracethreshold, 1, 2)
fig.append_trace(trace4, 1, 1)
fig.append_trace(trace3, 1, 2)


fig['layout'].update(height=400, width=1100, title='avg_VOC array & avg_PM array')


py.iplot(fig, filename='simple-subplot')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [43]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

d = parse_file("./DATA_DIR/"+txt_list[2])
df = pd.DataFrame({'my_dates':d['time_date'],'PM':d['pm'],'voc':d['voc']})
df['my_dates'] = pd.to_datetime(df['my_dates'])
df['days'] = df['my_dates'].dt.weekday_name


sunday=df.loc[df['days'] == 'Sunday']
monday=df.loc[df['days'] == 'Monday']
tuesday=df.loc[df['days'] == 'Tuesday']
wednesday=df.loc[df['days'] == 'Wednesday']
thuresday=df.loc[df['days'] == 'Thursday']
friday=df.loc[df['days'] == 'Friday']
saturday=df.loc[df['days'] == 'Saturday']

sundayval=np.average(sunday['PM'])
mondayval=np.average(monday['PM'])
tuesdayval=np.average(tuesday['PM'])
wednesdayval=np.average(wednesday['PM'])
thuresdayval=np.average(thuresday['PM'])
fridayval=np.average(friday['PM'])
saturdayval=np.average(saturday['PM'])

sundayval1=np.average(sunday['voc'])
mondayval1=np.average(monday['voc'])
tuesdayval1=np.average(tuesday['voc'])
wednesdayval1=np.average(wednesday['voc'])
thuresdayval1=np.average(thuresday['voc'])
fridayval1=np.average(friday['voc'])
saturdayval1=np.average(saturday['voc'])

trace5 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval1, mondayval1, tuesdayval1, wednesdayval1, thuresdayval1, fridayval1, saturdayval1],
  mode = 'lines+markers',
  name = 'avg_VOC array'
   
)
data = [trace5]


trace6 = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y = [sundayval, mondayval, tuesdayval, wednesdayval, thuresdayval, fridayval, saturdayval],
  mode = 'lines+markers',
  name = 'avg_PM array'
   
)
data = [trace6]

tracethreshold = go.Scatter(
  x = ['Sunday','Monday', 'Tuesday', 'Wednesday', 'Thuresday', 'Friday', 'Saturday'],
  y =  [200, 200, 200, 200, 200, 200, 200],
  mode = 'lines+markers',
  name = 'voc_threshold',
     line = dict(
       color = ('rgb(205, 12, 24)'),
       width = 4,
       dash = 'dash') # dash options include 'dash', 'dot', and 'dashdot'
)
data = [tracethreshold]

fig = tools.make_subplots(rows=1, cols=2)


fig.append_trace(tracethreshold, 1, 2)
fig.append_trace(trace6, 1, 1)
fig.append_trace(trace5, 1, 2)


fig['layout'].update(height=400, width=1100, title='avg_VOC array & avg_PM array')


py.iplot(fig, filename='simple-subplot')


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [49]:
fig = tools.make_subplots(rows=2, cols=2)

fig.append_trace(tracethreshold, 1, 2)
fig.append_trace(tracethreshold, 2, 2)
fig.append_trace(trace6, 1, 1)
fig.append_trace(trace5, 1, 2)
fig.append_trace(trace6, 2, 1)
fig.append_trace(trace5, 2, 2)

fig['layout'].update(height=800, width=1100, title='avg_VOC array & avg_PM array')


py.iplot(fig, filename='simple-subplot')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

